Let's check that your `GOOGLE_API_KEY` is set and, if not, you will be asked to enter it.

In [3]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

We will use Gemini's API key as it does not require a payment method upfront. OpenAI's key could be used as well but make sure you understand the limitations of the [free credits](https://community.openai.com/t/understanding-api-limits-and-free-tier/498517). You can see pricing for various models [here](https://openai.com/api/pricing/).

## Model selection

There are a few standard parameters that we can set with chat models. Two of the most common are:

* `model`: the name of the model
* `temperature`: the sampling temperature

`Temperature` controls the randomness or creativity of the model's output where low temperature (close to 0) is more deterministic and focused outputs. This is good for tasks requiring accuracy or factual responses. High temperature (close to 1) is good for creative tasks or generating varied responses. 

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

gemini20 = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
gemini25 = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

E0000 00:00:1759252835.315012 1243892 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1759252835.316400 1243892 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Chat models in LangChain have a number of [default methods](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface). For the most part, we'll be using:

* `stream`: stream back chunks of the response
* `invoke`: call the chain on an input

And, as mentioned, chat models take [messages](https://python.langchain.com/v0.2/docs/concepts/#messages) as input. Messages have:

* a **role** that describes who is saying the message
* a **content** related to the message or instructions sent to the role

In [5]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Iraitz")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
gemini20.invoke(messages)

AIMessage(content='Hello there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--05a61d7c-a8c1-411d-af42-e99231386eaa-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

We get an `AIMessage` response. Also, note that we can just invoke a chat model with a string. When a string is passed in as input, it is converted to a `HumanMessage` and then passed to the underlying model.


In [6]:
gemini25.invoke("hello world")

AIMessage(content='Hello there!\n\nWelcome to the world of programming (or just saying hello!). How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--45a9c14a-cb74-4087-8b0d-5c600283700e-0', usage_metadata={'input_tokens': 3, 'output_tokens': 55, 'total_tokens': 58, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 32}})

In [5]:
gemini20.invoke("hello world")

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--82d18d48-d77c-4655-8eb9-0c76644c9859-0', usage_metadata={'input_tokens': 2, 'output_tokens': 10, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

The interface is consistent across all chat models, so you can easily switch between models without changing the downstream code if you have strong preference for another provider.

## Search Tools

You'll also see [Tavily](https://tavily.com/) in the README, which is a search engine optimized for LLMs and RAG, aimed at efficient, quick, and persistent search results. It's easy to sign up and offers a generous free tier.

More on it's [documentation](https://docs.tavily.com/welcome) even though it is pretty straight forward.

In [6]:
from langchain_tavily import TavilySearch

tavily_search = TavilySearch(max_results=3)
search_docs = tavily_search.invoke("What is LangGraph?")

In [7]:
search_docs

{'query': 'What is LangGraph?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.datacamp.com/tutorial/langgraph-tutorial',
   'title': 'LangGraph Tutorial: What Is LangGraph and How to Use It?',
   'content': 'LangGraph is a library within the LangChain ecosystem that provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured and efficient manner. By managing the flow of data and the sequence of operations, LangGraph allows developers to focus on the high-level logic of their applications rather than the intricacies of agent coordination. Whether you need a chatbot that can handle various types of user requests or a multi-agent system that performs complex tasks, LangGraph provides the tools to build exactly what you need. LangGraph significantly simplifies the development of complex LLM applications by providing a structured framework for managing state and coordinating agent interactio

We can use this resource to include relevant information into our prompts or ground the response of the agent.